In [1]:
!pip install chromadb sentence-transformers datasets tqdm scikit-learn numpy seaborn pandas torch

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 4.3 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.3/18.3 MB 58.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 101.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.9/94.9 kB 10.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.4/491.4 kB 43.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 117.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 95.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 67.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 845.4 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.2 MB/s

In [1]:
# ================================================================
# Intent Detection – benchmark multiparâmetro
# ================================================================

import os, time, json
from pathlib import Path
from collections import defaultdict, Counter

import numpy as np
from datasets import load_dataset
from tqdm import tqdm
from sklearn.metrics import accuracy_score, f1_score

import chromadb
from chromadb.utils import embedding_functions

from pprint import pprint
import torch

# ---------- Config global ---------------------------------------------------
CHROMA_BASE_DIR = "./chroma_stores"
BATCH_SIZE      = 1_000
COLLECTION_NAME = "utterances"

MODELS = {
    "BGE": {
        "provider": "sentence-transformers",
        "name": "BAAI/bge-base-en-v1.5"
    },
    "BGE-Large": {
        "provider": "sentence-transformers",
        "name": "BAAI/bge-large-en-v1.5"
    },
    "MPNet": {
        "provider": "sentence-transformers",
        "name": "sentence-transformers/all-mpnet-base-v2"
    },
    "GTR-T5-Base": {
        "provider": "sentence-transformers",
        "name": "sentence-transformers/gtr-t5-base"
    },
    "GTR-T5-Large": {
        "provider": "sentence-transformers",
        "name": "sentence-transformers/gtr-t5-large"
    },
    "LaBSE": {
        "provider": "sentence-transformers",
        "name": "sentence-transformers/LaBSE"
    },
    "DistilRoBERTa": {
        "provider": "sentence-transformers",
        "name": "sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2"
    },
    "All-MiniLM-L12-v2": {
        "provider": "sentence-transformers",
        "name": "sentence-transformers/all-MiniLM-L12-v2"
    },
    "All-MiniLM-L6-v2": {
        "provider": "sentence-transformers",
        "name": "sentence-transformers/all-MiniLM-L6-v2"
    }
}

# ---------- Utils -----------------------------------------------------------
def initialize_embedding(cfg):
    if cfg["provider"] == "sentence-transformers":
        return embedding_functions.SentenceTransformerEmbeddingFunction(
            model_name=cfg["name"])
    if cfg["provider"] == "openai":
        return embedding_functions.OpenAIEmbeddingFunction(
            model_name=cfg["name"],
            api_key=os.environ["OPENAI_API_KEY"])
    raise ValueError("Provider não suportado.")

def get_client(model_key):
    Path(CHROMA_BASE_DIR).mkdir(exist_ok=True)
    return chromadb.PersistentClient(
        path=os.path.join(CHROMA_BASE_DIR, model_key))

def get_collection(client, emb_fn):
    return client.get_or_create_collection(
        COLLECTION_NAME,
        metadata={"hnsw:space": "cosine"},
        embedding_function=emb_fn)

def index_dataset(model_key):
    """Indexa o split 'train' uma única vez (idempotente)."""
    client = get_client(model_key)
    if client.list_collections():            # já existe
        return
    emb_fn = initialize_embedding(MODELS[model_key])
    coll   = get_collection(client, emb_fn)

    ds   = load_dataset("clinc_oos", "plus")
    docs = ds["train"]["text"]
    lbls = ds["train"].features["intent"].names
    metas = [{"intent": lbls[i]} for i in ds["train"]["intent"]]
    ids  = [f"u{i}" for i in range(len(docs))]

    print(f"[{model_key}] indexando {len(docs)} exemplos…")
    for s in tqdm(range(0, len(docs), BATCH_SIZE)):
        e = min(s + BATCH_SIZE, len(docs))
        coll.add(ids=ids[s:e], documents=docs[s:e], metadatas=metas[s:e])
    print(f"[{model_key}] pronto.")

# ---------- Preditor com desempate -----------------------------------------
def predict_intent(intents, dists, thr):
    if not intents:
        return "oos"
    votes     = Counter(intents)
    max_v     = max(votes.values())
    cands     = [i for i, v in votes.items() if v == max_v]
    if len(cands) > 1:                       # desempate pela média da distância
        mean = {i: np.mean([d for (it, d) in zip(intents, dists) if it == i])
                for i in cands}
        intent = min(mean, key=mean.get)
    else:
        intent = cands[0]
    if (1 - dists[0]) < thr:
        intent = "oos"
    return intent

# ---------- Avaliação -------------------------------------------------------
def evaluate_metrics(coll, k_list, thresholds, split="test"):
    ds   = load_dataset("clinc_oos", "plus")
    txts = ds[split]["text"]
    lbls = ds[split].features["intent"].names
    gold = [lbls[i] for i in ds[split]["intent"]]

    k_list      = sorted(k_list)
    thresholds  = sorted(thresholds)
    K_max       = k_list[-1]

    # armazenar predições
    preds = {(k, t): [] for k in k_list for t in thresholds}
    lat   = []

    for text in tqdm(txts, desc="avaliando"):
        t0  = time.time()
        res = coll.query(query_texts=[text], n_results=K_max,
                         include=["metadatas", "distances"])
        lat.append((time.time() - t0) * 1_000)
        intents = [m["intent"] for m in res["metadatas"][0]]
        dists   = res["distances"][0]

        for k in k_list:
            sub_int = intents[:k]
            sub_dis = dists[:k]
            for t in thresholds:
                preds[(k, t)].append(predict_intent(sub_int, sub_dis, t))

    # métricas
    metric_table = {k: {} for k in k_list}
    true_oos = [g == "oos" for g in gold]

    for k in k_list:
        for t in thresholds:
            pr = preds[(k, t)]
            acc = round(accuracy_score(gold, pr), 4)
            f1  = round(f1_score(gold, pr, average="macro"), 4)

            # recall da classe OOS
            pred_oos    = [p == "oos" for p in pr]
            acertos_oos = sum(1 for g, p in zip(true_oos, pred_oos) if g and p)
            oos_acerto  = round(acertos_oos / sum(true_oos), 4)

            metric_table[k][t] = {
                "accuracy"   : acc,
                "macro_f1"   : f1,
                "oos_acerto" : oos_acerto
            }

    latency = {
        "avg_ms": round(np.mean(lat), 2),
        "p50_ms": round(np.percentile(lat, 50), 2),
        "p95_ms": round(np.percentile(lat, 95), 2),
        "p99_ms": round(np.percentile(lat, 99), 2),
        "max_ms": round(np.max(lat), 2)
    }

    return {"latency_stats": latency, "metrics": metric_table}

# ---------- Benchmark multiparâmetro ---------------------------------------
def run_benchmark(model_list, k_list, thresholds, split="test"):
    results_all = {}
    for model_key in model_list:
        index_dataset(model_key)                          # idempotente
        coll = get_collection(get_client(model_key),
                              initialize_embedding(MODELS[model_key]))
        res  = evaluate_metrics(coll, k_list, thresholds, split)
        results_all[model_key] = res
    return results_all

In [ ]:
model_list = [
        "BGE",
        "BGE-Large",
        "MPNet",
        "GTR-T5-Base",
        "GTR-T5-Large",
        "LaBSE",
        "DistilRoBERTa",
        "All-MiniLM-L12-v2",
        "All-MiniLM-L6-v2"
    ]

# Valores progressivos de k e thresholds
k_list = [1, 3, 5, 10, 20, 100]
thresholds = [0.30, 0.40, 0.55, 0.70, 0.85]

results_all = run_benchmark(model_list, k_list, thresholds, split="test")
pprint(results_all)


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/94.6k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/52.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/777 [00:00<?, ?B/s]

...

Generating train split:   0%|          | 0/15250 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/3100 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/5500 [00:00<?, ? examples/s]

[BGE] indexando 15250 exemplos…


  6%|▋         | 1/16 [00:24<06:12, 24.82s/it]

In [ ]:
import pandas as pd

# Converte o dicionário results_all em DataFrame
data = []

for model, res in results_all.items():
    lat = res["latency_stats"]
    for k, thr_dict in res["metrics"].items():
        for thr, vals in thr_dict.items():
            data.append({
                "model": model,
                "k": k,
                "threshold": thr,
                "accuracy": vals["accuracy"],
                "macro_f1": vals["macro_f1"],
                "oos_acerto": vals["oos_acerto"],
                "latency_avg_ms": lat["avg_ms"]
            })

df = pd.DataFrame(data)


In [ ]:
df[(df["k"] == 10) & (df["threshold"] == 0.55)]\
    .sort_values("accuracy", ascending=False)\
    .reset_index(drop=True)


In [ ]:
df[df["model"] == "BGE-Large"]\
    .sort_values(["k", "threshold"])\
    .reset_index(drop=True)


In [ ]:
df[df["oos_acerto"] > 0.9]\
    .sort_values(["accuracy"], ascending=False)


In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

sns.lineplot(data=df[df["model"] == "MPNet"],
             x="threshold", y="accuracy", hue="k")
plt.title("Accuracy por Threshold (MPNet)")
plt.grid(True)
plt.show()
